In [287]:
import requests
import os
from dotenv import load_dotenv

In [288]:
url = ""
load_dotenv()
API_KEY = os.getenv("API_KEY")
header = {"Authorization":API_KEY}
data = {"item":""}
incoming = requests.get(url,headers=header,data=data)
if incoming.status_code==200:
    print(incoming.json())

MissingSchema: Invalid URL '': No scheme supplied. Perhaps you meant https://?

Translating text from German to English


In [371]:
from transformers import MarianMTModel, MarianTokenizer

In [372]:
model_name = 'Helsinki-NLP/opus-mt-de-en'
tokenizer = MarianTokenizer.from_pretrained(model_name)
model = MarianMTModel.from_pretrained(model_name)


c:\Users\muthu\Documents\bfh\BFH-offenburg-burda\bfh\Lib\site-packages\transformers\models\marian\tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [373]:
def translate(text):
    # Tokenize the input text
    tokenized_text = tokenizer.prepare_seq2seq_batch([text], return_tensors='pt')
    
    # Perform the translation
    translation = model.generate(**tokenized_text)
    
    # Decode the translated text
    translated_text = tokenizer.decode(translation[0], skip_special_tokens=True)
    
    return translated_text

In [374]:
import json
import re
from typing import List, Dict, Any, Iterable

# --- basic helpers ---
def clean_spaces(s: str) -> str:
    """Collapse multiple spaces and trim."""
    return re.sub(r"\s+", " ", s).strip()

def join_ingredient_parts(item: dict) -> str:
    """Build a readable ingredient string including prefix/suffix if available."""
    prefix = item.get("prefix") or ""
    name = (item.get("ingredient") or {}).get("name", "") or ""
    suffix = item.get("suffix") or ""
    text = clean_spaces(f"{prefix}{name}")
    if suffix:
        text = clean_spaces(f"{text} {suffix}")
    return text
def join_quantity(item:dict):
    quantity = item.get("quantity") or ""
    unit = (item.get("unit") or {}).get("name","") or ""
    text = clean_spaes(f"{quantity}{unit}")
    return text

def iter_ingredients(recipe: dict) -> Iterable[dict]:
    """Iterate through all ingredient items in a recipe."""
    for block in recipe.get("ingredientBlocks", []):
        for it in block.get("ingredients", []):
            yield it
    # some recipes also have ingredients listed inside content sections
    for section in recipe.get("content", []):
        for it in section.get("ingredients", []) or []:
            yield it

def dedup_keep_order(items: List[str]) -> List[str]:
    """Remove duplicates but keep the original order."""
    seen = set()
    result = []
    for x in items:
        key = x.lower()
        if key not in seen:
            seen.add(key)
            result.append(x)
    return result


# --- load the JSON file ---
with open("einfachbacken_export_200_recipes_1.json", "r", encoding="utf-8") as f:
    data = json.load(f)

recipes: List[Dict[str, Any]] = data.get("data", {}).get("recipeExport", [])


# --- single recipe ---
def ingredients_for_recipe(recipes: List[dict], recipe_name: str, dedup: bool = True) -> List[str]:
    recipe = next((r for r in recipes if r.get("name") == recipe_name), None)
    if not recipe:
        return []
    items = [join_ingredient_parts(it) for it in iter_ingredients(recipe)]
    
    return dedup_keep_order(items) if dedup else items
# --- all recipes combined ---
def ingredients_global(recipes: List[dict], dedup: bool = True) -> List[str]:
    items = []
    for r in recipes:
        items.extend(join_ingredient_parts(it) for it in iter_ingredients(r))
    return dedup_keep_order(items) if dedup else items


# --- per-recipe dictionary ---
def ingredients_per_recipe(recipes: List[dict], dedup: bool = True) -> Dict[str, List[str]]:
    result = {}
    for r in recipes:
        items = [join_ingredient_parts(it) for it in iter_ingredients(r)]
        result[r.get("name", "Unnamed")] = dedup_keep_order(items) if dedup else items
    return result


# --- example usage ---
one_recipe = ingredients_for_recipe(recipes, "Veganes Stollenkonfekt")
print("Single recipe:", one_recipe)

all_ingredients = ingredients_global(recipes)
print("Global:", all_ingredients[:20], f"... ({len(all_ingredients)} total)")

by_recipe = ingredients_per_recipe(recipes)
print("Recipes found:", list(by_recipe.keys())[:3])

Single recipe: ['Marzipan', 'Weizenmehl (Type 405)', 'Trockenhefe', 'Zucker', 'Vanillezucker', 'gemahlenen Mandeln', 'Stollengewürz', 'pflanzliche Milch', 'vegane Butter (Zimmertemperatur)', 'Rosinen', 'Orangeat', 'Zitronat', 'Saft (oder Rum)', 'Etwas Mehl zum Arbeiten', 'Puderzucker', 'Vanillinzucker', 'gemahlene Mandeln', 'veganeButter (Zimmertemperatur)', 'EtwasMehl zum Arbeiten']
Global: ['Marzipan', 'Weizenmehl (Type 405)', 'Trockenhefe', 'Zucker', 'Vanillezucker', 'gemahlenen Mandeln', 'Stollengewürz', 'pflanzliche Milch', 'vegane Butter (Zimmertemperatur)', 'Rosinen', 'Orangeat', 'Zitronat', 'Saft (oder Rum)', 'Etwas Mehl zum Arbeiten', 'Puderzucker', 'Vanillinzucker', 'gemahlene Mandeln', 'veganeButter (Zimmertemperatur)', 'EtwasMehl zum Arbeiten', 'Ei (Größe M)'] ... (743 total)
Recipes found: ['Veganes Stollenkonfekt', 'Wickeltorte', 'Orangen-Dessert mit Joghurt']


In [375]:
import re
import json
from typing import List, Dict, Any, Iterable

# --- basic helpers (same as before, with the typo fixed) ---
def clean_spaces(s: str) -> str:
    return re.sub(r"\s+", " ", str(s)).strip()

def join_ingredient_parts(item: dict) -> str:
    prefix = item.get("prefix") or ""
    name = (item.get("ingredient") or {}).get("name", "") or ""
    suffix = item.get("suffix") or ""
    text = clean_spaces(f"{prefix} {name}")
    if suffix:
        text = clean_spaces(f"{text} {suffix}")
    return text

def extract_quantity_unit(item: dict) -> str:
    q = item.get("quantity")
    q_str = "" if q in (None, "") else str(q)
    unit_obj = item.get("unit") or {}
    unit = unit_obj.get("name") or unit_obj.get("short") or unit_obj.get("abbreviation") or ""
    return clean_spaces(f"{q_str} {unit}")  # e.g. "200 g" or "" if none

def join_quantity(item: dict) -> str:
    return extract_quantity_unit(item)

def iter_ingredients(recipe: dict) -> Iterable[dict]:
    for block in recipe.get("ingredientBlocks", []):
        for it in block.get("ingredients", []):
            yield it
    for section in recipe.get("content", []) or []:
        for it in section.get("ingredients", []) or []:
            yield it

# --- load your JSON export (adjust filename as needed) ---
with open("einfachkochen_export_800_recipes_1.json", "r", encoding="utf-8") as f:
    data = json.load(f)
recipes: List[Dict[str, Any]] = data.get("data", {}).get("recipeExport", [])

# --- NEW: dict of {ingredient: quantity} for ONE recipe ---
def ingredients_qty_dict_for_recipe(recipes: List[dict], recipe_name: str) -> Dict[str, str]:
    """
    Returns: { "Sugar": "200 g", "Milk": "100 ml", "Egg": "2" , ... }
    If an ingredient appears multiple times, the last one wins.
    """
    recipe = next((r for r in recipes if r.get("name") == recipe_name), None)
    if not recipe:
        return {}
    out: Dict[str, str] = {}
    for it in iter_ingredients(recipe):
        ing = join_ingredient_parts(it)
        qty = join_quantity(it)  # already "number unit" or ""
        out[ing] = qty
    return out

# --- OPTIONAL: dict of recipe -> {ingredient: quantity} for ALL recipes ---
def ingredients_qty_dicts_all(recipes: List[dict]) -> Dict[str, Dict[str, str]]:
    out: Dict[str, Dict[str, str]] = {}
    for r in recipes:
        rname = r.get("name", "Unnamed")
        d: Dict[str, str] = {}
        for it in iter_ingredients(r):
            ing = join_ingredient_parts(it)
            qty = join_quantity(it)
            d[ing] = qty
        out[rname] = d
    return out

# --- example usage ---
one_recipe_map = ingredients_qty_dict_for_recipe(recipes, "Klassisches Gulasch")
print(one_recipe_map)  # {'Mehl': '300 g', 'Zucker': '120 g', ...}

all_recipes_map = ingredients_qty_dicts_all(recipes)
print(list(all_recipes_map.keys())[0:10])


{'Zwiebel': '600 g', 'etwas Butterschmalz (oder Sonnenblumenöl) zum Anbraten': '', 'gemischtes Gulaschfleisch': '600 g', 'Salz': '0.5 TL', 'Pfeffer': '0.5 TL', 'Zucker': '1 TL', 'Rotwein': '300 ml', 'Paprikapulver (edelsüß)': '2 TL', 'Paprikapulver (rosenscharf)': '1 TL', 'getrockneter Majoran': '1 TL', 'gemahlener Kümmel': '1 TL', 'Wasser': '1 Liter', 'Essig': '1 EL', 'Petersilie': '1 Bd.', 'etwas Butterschmalz (oder Sonnenblumenöl)': '', 'etwas Salz': ''}
['Klassisches Gulasch', 'Smørrebrød', 'Gurken-Sushi', 'Pfannkuchen-Raclette', 'Steak au four', 'Spitzkohl aus dem Ofen', 'Pfirsich-Dessert ', 'Piccata Milanese', 'Würzfleisch', 'Tortellini in Schinken-Sahne-Soße']


In [376]:
translated_texts_all = []

In [377]:
for src_texts in one_recipe_map.keys():
    inputs = tokenizer(src_texts, return_tensors="pt", padding=True, truncation=True)
    translated = model.generate(**inputs)

    # Decode and print result
    translated_texts = tokenizer.batch_decode(translated, skip_special_tokens=True)
    # print(translated_texts[0])
    translated_texts_all.append(translated_texts[0])

In [378]:
old_keys = list(one_recipe_map)  # preserves order of insertion
new_keys = translated_texts_all  # same length as old_keys

# 1) safest: build a new dict
renamed = {new: one_recipe_map[old] for old, new in zip(old_keys, new_keys)}

In [379]:
one_recipe_map = renamed

In [380]:
import re

pck = 7  # adjust if you like
TL_G = 5
EL_G = 15
def quantity_conversion(quantity: str):
    s = quantity.strip().lower()
    # grab the first number (works for "100 g", "100g", "1.5 kg", "250ml")
    m = re.search(r'([\d.,]+)', s)
    if not m:
        return None
    n = float(m.group(1).replace(',', '.'))  # <- n is just the number part

    # convert to kg (simple assumptions)
    if 'kg' in s:
        convert = n
    elif 'g' in s:
        convert = n / 1000.0
    elif 'ml' in s:
        convert = n / 1000.0   # assume 1 ml ~ 1 g ~ 0.001 kg
    elif 'l' in s:
        convert = n 
    elif " tl" in s or s.endswith("tl"):
        convert = (n * TL_G) / 1000.0
    elif " el" in s or s.endswith("el"):
        convert = (n * EL_G) / 1000.0
    elif " pck" in s:
        convert = (n * pck) / 1000.0           # assume 1 L ~ 1 kg
    else:
        convert = n * 0.1         # unknown unit

    return convert


In [381]:
import pandas as pd
from rapidfuzz import process, fuzz


In [382]:
df = pd.read_csv(r"C:\Users\muthu\Documents\bfh\BFH-offenburg-burda\final_co2_emissions_only_germany.csv",encoding="cp1252")



In [383]:
df = df.rename(columns={'Emissions (CO2 eq/kg)':'Emissions'})

In [384]:
def normalize(word):
    if isinstance(word,str):

        word = word.lower()
        word = word.split(",",1)[0]
        
    else:
        word = ""
    return word

df["norm"] = df["Product"].map(normalize)


In [385]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

In [386]:
model = SentenceTransformer("all-MiniLM-L6-v2")
name_vecs = model.encode(df["norm"].tolist(), normalize_embeddings=True).astype("float32")
index = faiss.IndexFlatIP(name_vecs.shape[1])   # inner product == cosine for normalized vectors
index.add(name_vecs)

In [409]:
def embedding_lookup(name: str, addition, cos_cutoff=0.50):
    q = model.encode([normalize(name)], normalize_embeddings=True).astype("float32")
    D, I = index.search(q, 1)  # top-1
    sim = float(D[0][0])
    if sim < cos_cutoff:
        return None, 0.0, addition  # no match → 0 CO₂

    idx = int(I[0][0])
    row = df.iloc[idx]
    kg_co2 = float(row["Emissions"])

    q_co2 = quantity_conversion(one_recipe_map[name])
    if q_co2 is None:
        return None, 0.0, addition  # missing quantity → 0 CO₂

    total = kg_co2 * q_co2
    addition += total

    return row["norm"], total, addition

In [411]:
addition = 0
for i in list(one_recipe_map):
    
    p,t,a = embedding_lookup(i,addition=addition)
    addition = a
    if p is None:
        p = i
        t = 0
    print(p,t,a)

onion 0.15 0.15
a little butter malt (or sunflower oil) to fry 0 0.15
minced meat (beef 
and pork) 3.3539999999999996 3.5039999999999996
salt 0.25 3.7539999999999996
pepper 0.32 4.074
sugar 0.6 4.6739999999999995
red wine 0.432 5.106
sweet pepper 1.28 6.386
pepper 0.64 7.026
marjoram 0.9 7.926
Ground caraway 0 7.926
mineral water 0.2 8.126
vinegar 0.46 8.586
parsley 0.04000000000000001 8.626
a little butter malt (or sunflower oil) 0 8.626
a little salt 0 8.626


In [403]:
addition

0

In [171]:
translated_texts_all

['Marzipan',
 'Wheat flour (type 405)',
 'Dry yeast',
 'Sugar',
 'Vanilla sugar',
 'Ground almonds',
 'Stollen seasoning',
 'vegetable milk',
 'vegan butter (room temperature)',
 'Rice',
 'Orangeate',
 'Citronate',
 'Juice (or rum)',
 'Some flour to work with',
 'Powder sugar',
 'Vanillin sugar',
 'Ground almonds',
 'vegan butter (room temperature)',
 'Something to work with']